In [1]:
## 패키지 선언
from selenium import webdriver
from bs4 import BeautifulSoup as BS
import pandas as pd

In [2]:
## 내가 조종할 크롬 노예 생성 
driver = webdriver.Chrome()

In [3]:
url = "https://www.disneyplus.com/ko-kr/movies/9f7c38e5-41c3-47b4-b99e-b5b3d2eb95d4"

In [4]:
driver.get(url)

In [5]:
## 클릭에 필요한 패키지 선언
from selenium.webdriver.common.by import By

# 내가 클릭하고 싶은 위치를 selector로 Copy
target = "body > header > nav.nav.pre-sticky > ul > li > a"

## selector 로 복사해왔기 때문에 설정 , 위치에 요소를 찾고 클릭
driver.find_element(By.CSS_SELECTOR, target).click()

In [6]:
# 아이디와 비밀번호
email = "@@@"
password = "@@@"

In [7]:
# 아이디 양식 찾기
email_input = driver.find_element(By.ID, "email") 

# 아이디와 비밀번호 입력
email_input.send_keys(email)

#다음 클릭 
driver.find_element(By.CSS_SELECTOR, "#loginEmail > div:nth-child(4) > button").click()

In [8]:
# 비밀번호 양식 찾기
password_input = driver.find_element(By.ID, "password")  # 실제 비밀번호 입력 필드의 ID로 변경해 주세요

# 비밀번호 입력
password_input.send_keys(password)

# 로그인 클릭 
driver.find_element(By.CSS_SELECTOR, "#password-continue-login").click()

In [9]:
# 프로필 선택
driver.find_element(By.CSS_SELECTOR, "#remove-main-padding_index > div > div > section > ul > div:nth-child(1) > div > div").click()

In [10]:
import time

# 스크롤 계속 내리기
while True:
    # 현재 스크롤 높이 계산
    scroll_height = driver.execute_script("return document.body.scrollHeight")

    # 스크롤 내리기
    driver.execute_script(f"window.scrollTo(0, {scroll_height});")

    # 새로운 내용이 로드되길 기다림
    time.sleep(2)

    # 새로운 스크롤 높이 확인
    new_scroll_height = driver.execute_script("return document.body.scrollHeight")

    # 새로운 스크롤 높이가 이전과 동일하면 스크롤이 끝난 것으로 판단하고 종료
    if new_scroll_height == scroll_height:
        break

In [15]:
link_element = driver.find_element(By.CSS_SELECTOR, "#section_index > article > div:nth-child(2) > div > div > div > div > div:nth-child(1)")

# <a> 요소를 클릭합니다.
link_element.click()

In [16]:
displus_list = []

In [18]:
for num in range(0,1124):
    displus_dict={}
    print(num)
    bs = BS(driver.page_source)
    driver.find_element(By.CSS_SELECTOR, f"#section_index > article > div:nth-child(2) > div > div > div > div > div:nth-child({num})").click()

    time.sleep(2)
    ## 페이지 소스 
    bs2 = BS(driver.page_source)

    ## 포스터
    displus_dict['img'] = bs.findAll("div",class_="gv2-asset")[num-1].find("img")['src']

    ## 제목 
    displus_dict['title'] = bs2.find("h1",class_="h3 padding--bottom-6 padding--right-6 text-color--primary").text

    ## 장르
    displus_dict['genre'] = bs2.findAll("p",class_="body-copy margin--0 text-color--primary")[4].text
    
    ## 줄거리 
    displus_dict['overview'] = bs2.find("p",class_="margin--0 body-copy body-copy--large text-color--primary").text

    ## 개봉연도
    displus_dict['open'] = bs2.findAll("p",class_="body-copy margin--0 text-color--primary")[3].text
    
    try:
        ## 감독
        displus_dict['director'] = bs2.findAll("p",class_="body-copy margin--0 text-color--primary")[5].text
    except:
        displus_dict['director'] = "NULL"

    try:    
        ## 배우
        displus_dict['actor'] = [x .text for x in bs2.findAll("p",class_="body-copy margin--0 text-color--primary")[6:]]
    except:
        displus_dict['actor'] = "NULL"
    ## 러닝타임
    displus_dict['runtime'] = bs2.find("div",class_="sc-AqAhp lhwxfs").find("p", class_="body-copy margin--0 text-color--primary").text
    
    displus_list.append(displus_dict)
    driver.back()
     
    time.sleep(2)

388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637


In [19]:
disney = pd.DataFrame(displus_list)

In [20]:
disney.to_csv("./disneymovies.csv")

In [21]:
disney

,img,title,genre,overview,open,director,actor,runtime
0,https://prod-ripcut-delivery.disney-plus.net/v...,내가 널 사랑할 수 없는 10가지 이유,"로⁠맨⁠스, 로⁠맨⁠틱 코⁠미⁠디, 성⁠장⁠ ⁠스⁠토⁠리, 코⁠미⁠디","멋진 청춘의 출연진과 함께하는 유쾌한 코미디! 전학 온 첫 날, 카메론(조셉 고든-...",1999,길 정거,"[히스 레저, 줄리아 스타일스, 조셉 고든 레빗, 라리사 올리닉, 데이비드 크럼홀츠...",1시간 39분
1,https://prod-ripcut-delivery.disney-plus.net/v...,101마리의 달마시안 개,"가⁠족, 애⁠니⁠메⁠이⁠션, 액⁠션⁠ ⁠어⁠드⁠벤⁠처","월트 디즈니의 오리지널 고전은 귀여운 동물들의 잊을 수 없는 이야기와 웃음, 모험으...",1961,"울프강 레이더먼, 해밀턴 S. 루스케, 클라이드 제로니미","[로드 테일러, J. 팻 오말리, 베티 루 거슨, 마사 웬트워스, 벤 라이트, 케이...",1시간 22분
2,https://prod-ripcut-delivery.disney-plus.net/v...,101 달마시안,"가⁠족, 동⁠물/자⁠연, 액⁠션⁠ ⁠어⁠드⁠벤⁠처",컴퓨터 게임 디자이너인 로저(Roger: 제프 다니엘즈 분)에게는 퐁고라는 이름의 ...,1996,스티븐 헤렉,"[글렌 클로즈, 제프 대니얼스, 조엘리 리처드슨, 조앤 플로라이트, 휴 로리, 마크...",1시간 44분
3,https://prod-ripcut-delivery.disney-plus.net/v...,101 달마시안 2,"애⁠니⁠메⁠이⁠션, 액⁠션⁠ ⁠어⁠드⁠벤⁠처",퐁고와 퍼디타의 새끼 강아지 패치는 101마리 중 한 마리로 사는 삶에는 만족하지 ...,2003,"짐 카메루드, 브라이언 스미스","[배리 보스트윅, 제이슨 알렉산더, 마틴 쇼트, 바비 록우드, 수잔느 블레이크슬리,...",1시간 18분
4,https://prod-ripcut-delivery.disney-plus.net/v...,102 달마시안,"가⁠족, 동⁠물/자⁠연, 액⁠션⁠ ⁠어⁠드⁠벤⁠처",애완견을 납치한 죄로 감옥에서 3년이라는 세월을 보낸 크루엘라 드 빌은 모피에는 손...,2000,케빈 리마,"[글렌 클로즈, 이안 그루퍼드, 앨리스 에반스, 팀 매키너니, 이언 리처드슨, 제라...",1시간 45분
...,...,...,...,...,...,...,...,...
1118,https://prod-ripcut-delivery.disney-plus.net/v...,좀비스 2,"로⁠맨⁠스, 성⁠장⁠ ⁠스⁠토⁠리, 판⁠타⁠지, 뮤⁠지⁠컬","획기적인 변화가 일어난 학기가 지나간 후, 제드와 애디슨은 학교와 마을에 화합을 가...",2020,폴 호엔,"[마일로 만하임, 멕 도넬리, 트레버 토르즈만, 카일 러셀, 칼라 제프리, 챈들러 키니]",1시간 26분
1119,https://prod-ripcut-delivery.disney-plus.net/v...,좀비스 3,"성⁠장⁠ ⁠스⁠토⁠리, SF, 판⁠타⁠지, 뮤⁠지⁠컬","올해는 제드와 애디슨에게 시브룩에서의 마지막 해이고, 모든 것이 더할 나위 없이 좋...",2022,Paul Hoen,"[Milo Manheim, Meg Donnelly, Trevor Tordjman, ...",1시간 30분
1120,https://prod-ripcut-delivery.disney-plus.net/v...,조이의 비밀 앱,"성⁠장⁠ ⁠스⁠토⁠리, SF, 코⁠미⁠디","엄마가 재혼하면서 조이는 지저분하고 정신 사나운 집에서 시끄러운 새아빠, 그리고 사...",2014,피터 드루이즈,"[젠다야, 채넬 펠로소, 스펜서 볼드먼, 에밀리아 맥카시, 애덤 디마르코, 윌리엄 ...",1시간 36분
1121,https://prod-ripcut-delivery.disney-plus.net/v...,좀비스 with 빅 시티 그린,"가⁠족, 코⁠미⁠디, 애⁠니⁠메⁠이⁠션, 뮤⁠지⁠컬","그린 가족이 한자리에 모여 ""좀비스"" 3부작을 추억하는 특집 버라이어티 쇼를 진행한...",2022,사브리나 알베르게티,"[맷 코넷, 크리스 호튼, 아르테미스 페바니, 멕 도넬리, 밥 졸스, 지노 로빈슨]",23분


In [11]:
dip = BS(driver.page_source).findAll("div",class_="gv2-asset").find("img")['src']

In [12]:
dip[0].find("div",class_="sc-bdVaJa hRKcBi image-container")['alt']

'내가 널 사랑할 수 없는 10가지 이유'

In [13]:
len(dip)

1123

# 디즈니 시리즈물 가져오기

In [3]:
ser_url ="https://www.disneyplus.com/ko-kr/series/53adf843-491b-40ae-9b46-bccbceed863b"

In [4]:
driver.get(ser_url)

In [19]:
import time

# 스크롤 계속 내리기
while True:
    # 현재 스크롤 높이 계산
    scroll_height = driver.execute_script("return document.body.scrollHeight")

    # 스크롤 내리기
    driver.execute_script(f"window.scrollTo(0, {scroll_height});")

    # 새로운 내용이 로드되길 기다림
    time.sleep(2)

    # 새로운 스크롤 높이 확인
    new_scroll_height = driver.execute_script("return document.body.scrollHeight")

    # 새로운 스크롤 높이가 이전과 동일하면 스크롤이 끝난 것으로 판단하고 종료
    if new_scroll_height == scroll_height:
        driver.execute_script("window.scrollTo(0,0);")
        break

# 줄거리

In [23]:
bs.find("p",class_="margin--0 body-copy body-copy--large text-color--primary").text

'오직 승리만을 추구하는 사상 최강의 야구팀이 탄생했다.\n"우리보다 최강인 팀은 절대 없을 겁니다."\n \n한국을 대표하는 최강 라인업으로 구성된 프로야구팀에 대적할 만한 11번째 구단 최강 몬스터즈와 전국의 야구 강팀이 펼치는 양보 없는 대결!\n\n야구에 미친 자들의 모든 걸 건 진짜 승부가 시작된다.'

# 러닝 타임

In [24]:
bs.findAll("div", class_="sc-AqAhp lhwxfs")[0].find("p", class_="body-copy margin--0 text-color--primary").text

'2시간'

# 공개일

In [25]:
bs.findAll("div", class_="sc-AqAhp lhwxfs")[1].find("p", class_="body-copy margin--0 text-color--primary").text

'2022'

# 장르

In [32]:
bs.findAll("div", class_="sc-AqAhp lhwxfs")[2].find("p", class_="body-copy margin--0 text-color--primary").text.replace("\u2060","")

'리얼리티, 스포츠'

# 제목

In [33]:
bs.find("h1",class_="h3 padding--bottom-6 padding--right-6 text-color--primary").text

'최강야구'

# 시즌 개수

In [39]:
import re

pattern = r"시즌\s[0-9]개"

In [86]:
re.search(pattern, bs2.find("div", class_="sc-kDgGX gDKweb").find("span").text).group()

AttributeError: 'NoneType' object has no attribute 'find'

# 출연

In [54]:
[x.text.replace(" ","") for x in bs.find("div",class_="sc-exdmVY hWjZRq").findAll("p", class_="body-copy margin--0 text-color--primary")]

['이승엽', '박용택', '송승준', '이택근', '정성훈', '심수창']

In [65]:
displus_list= []

In [90]:
import re

pattern = r"시즌\s[0-9]+개"
for num in range(1,820):
    displus_dict={}
    print(num)
    bs = BS(driver.page_source)
    driver.find_element(By.CSS_SELECTOR, f"#section_index > article > div:nth-child(2) > div > div > div > div > div:nth-child({num})").click()

    time.sleep(3)
    ## 페이지 소스 
    bs2 = BS(driver.page_source)

    ## 포스터
    displus_dict['img'] = bs.findAll("div",class_="gv2-asset")[num-1].find("img")['src']

    ## 제목 
    displus_dict['title'] = bs.findAll("div",class_="sc-bdVaJa hRKcBi image-container")[num-1]['alt']
    
    ## 시즌 개수
    displus_dict['series'] = re.search(pattern, bs2.find("div", class_="sc-kDgGX gDKweb").find("span").text).group()

    try:
        ## 장르
        displus_dict['genre'] = bs2.findAll("div", class_="sc-AqAhp lhwxfs")[2].find("p", class_="body-copy margin--0 text-color--primary").text.replace("\u2060","")
    except:
        pass
    ## 줄거리 
    displus_dict['overview'] = bs2.find("p",class_="margin--0 body-copy body-copy--large text-color--primary").text

    ## 개봉연도
    displus_dict['open'] = bs2.findAll("div", class_="sc-AqAhp lhwxfs")[1].find("p", class_="body-copy margin--0 text-color--primary").text
    
    try:
        ## 출연
        displus['actors'] = [x.text.replace(" ","") for x in bs2.find("div",class_="sc-exdmVY hWjZRq").findAll("p", class_="body-copy margin--0 text-color--primary")]
    except:
        pass

    ## 러닝타임
    displus_dict['runtime'] = bs2.findAll("div", class_="sc-AqAhp lhwxfs")[0].find("p", class_="body-copy margin--0 text-color--primary").text
    
    displus_list.append(displus_dict)
    driver.back()
     
    time.sleep(2)

650


WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=114.0.5735.133)
Stacktrace:
0   chromedriver                        0x00000001013cff48 chromedriver + 4226888
1   chromedriver                        0x00000001013c84f4 chromedriver + 4195572
2   chromedriver                        0x000000010100cd68 chromedriver + 281960
3   chromedriver                        0x0000000100ff5db4 chromedriver + 187828
4   chromedriver                        0x0000000100ff5b9c chromedriver + 187292
5   chromedriver                        0x000000010107e4a4 chromedriver + 746660
6   chromedriver                        0x000000010103bf1c chromedriver + 474908
7   chromedriver                        0x000000010103cef4 chromedriver + 478964
8   chromedriver                        0x000000010139159c chromedriver + 3970460
9   chromedriver                        0x00000001013956f0 chrome